
(tutorials-hubbard-base)=

# Computing Hubbard parameters

In this tutorial you will learn how to calculate the Hubbard parameters step by step using `aiida-hubbard`.

We can divide this goal in three phases:

* __Define the manifolds__: define the target Hubbard manifolds via the {{ hubbard_structure }}
* __SCF ground-state__: calculate the ground-state using the {py:class}`~aiida_quantumespresso.workflows.pw.base.PwBaseWorkChain`
* __DFPT calculation__: use the {py:class}`~aiida_hubbard.workflow.hp.base.HpBaseWorkChain` to do a self-consistent perturbation calculation to predict the Hubbard parameters.

In this tutorial we will make use of the silicon structure to give you an overall understanding of the usage of the package.
If you are interested in more advanced features, please have a look at the [next tutorial](./2_parallel_hubbard.ipynb) or to the [how tos](howto).

Let's get started!


## Defining the target manifold through the `HubbardStructureData`

The Hubbard correction is a corrective term that is added to the Hamiltonian of a system
which suffers from great __self-interaction errors__. This is usually the case for transition
metals on their _d_ manifolds. An extra correction to account for the hybridization can be accounted
for with the ligands, typically belonging to the _p_ element group. Such interaction needs to be
localized in space. This is the reason why we need to define the __projectors__. Quantum ESPRESSO
allows you to define different type of projections $| \phi^I_m \rangle$ ($m$ orbital quantum number, $I$ atom in cell). Currently, the __ortho-atomic__ projectors
are the most accurate ones implemented. 

Still, we need to ask the program on _which atoms_ $I$ and _which manifolds_ $m$ to project and correct for this
self-interaction.

Since manifolds and atoms belong to the structure, then you need to definet them together as an {{ hubbard_structure }}.

In the following, we take LiCoO{sub}`2` as example, and we suppose we want to target the _3d_ orbitals of cobalt and the intersite interaction between _2p_ of oxygen and _3d_ of cobalt.

```{note}
By default we set ortho-atomic projectors and we use the Dudarev formulation.
```

In [ ]:
from local_module import load_temp_profile

# If you download this file, you can run it with your own profile.
# Put these lines instead:
# from aiida import load_profile
# load_profile()
data = load_temp_profile(
    name="hubbard-base-tutorial",
    add_computer=True,
    add_pw_code=True,
    add_hp_code=True,
    add_sssp=True,
)

Let's define the {{ hubbard_structure }}:

:::{note}
:class: dropdown

If you already have a {py:class}`aiida.orm.StructureData`, you can load the structure information in `HubbardStructureData` as follows:

```python
my_structure = load_node(IDENTIFIER)
hubbard_structure = HubbardStructureData.from_structure(my_structure)
```
:::

In [ ]:
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

a, b, c, d = 1.40803, 0.81293, 4.68453, 1.62585
cell = [[a, -b, c], [0.0, d, c], [-a, -b, c]]
sites = [
    ['Co', 'Co', (0, 0, 0)],
    ['O',   'O', (0, 0, 3.6608)], 
    ['O',   'O', (0, 0, 10.392)], 
    ['Li', 'Li', (0, 0, 7.0268)],
]
hubbard_structure = HubbardStructureData(cell=cell, sites=sites)
hubbard_structure.initialize_onsites_hubbard("Co", "3d")
hubbard_structure.initialize_intersites_hubbard("Co", "3d", "O", "2p")
hubbard_structure.store()

Let's visualize what will be print in the Hubbard card of Quantum ESPRESSO.

In [ ]:
from aiida_quantumespresso.utils.hubbard import HubbardUtils
print(HubbardUtils(hubbard_structure).get_hubbard_card())

As you can see, the desired interactions have been initialized correctly. 
This is important because `hp.x` needs to know which atoms need to be perturbed. 
As you will see later, `hp.x` will take care of adding the remaining interactions with neighbouring atoms.

:::{important}
When you use your own structures, make sure to have your 'Hubbard atoms' first in the list of atoms.
This is due to the way the `hp.x` routine works internally, requiring those to be first.
You can simply do this with the following snippet (IF THE NODE IS YET NOT STORED!):

```python
from aiida_quantumespresso.utils.hubbard import HubbardUtils
HubbardUtils(hubbard_structure).reorder_atoms
```
:::

## Calculating the SCF ground-state

Now that we have defined the structure, we can calculate its ground-state via an SCF using the `PwBaseWorkChain`.
We can fill the inputs of the builder of the `PwBaseWorkChain` through the `get_builder_from_protocol()` method.

In [ ]:
from aiida.engine import run_get_node
from aiida.orm import KpointsData
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_quantumespresso.common.types import ElectronicType
kpoints = KpointsData()
kpoints.set_kpoints_mesh([2,2,2])

builder = PwBaseWorkChain.get_builder_from_protocol(
    code=data.pw_code, # modify here if you downloaded the notebook
    structure=hubbard_structure,
    protocol="fast",
    electronic_type=ElectronicType.INSULATOR,
    overrides={"kpoints":kpoints, "clean_workdir":False}
)
results, pw_node = run_get_node(builder)
results

As you can notice from the results, the workchain (actually, the `PwCalculation`!) has a `remote_folder` output.
This is what we need in order to run the `HpBaseWorkChain`. 

## DFPT calculation of Hubbard parameters

We can perturb the ground-state previously found to compute the Hubbard parameters.
Here we will need to use the `HpBaseWorkChain`, and link the `remote_folder` previously produced via the `parent_scf` input.

In [ ]:
from aiida.orm import Dict
from aiida_hubbard.workflows.hp.base import HpBaseWorkChain

qpoints = KpointsData()
qpoints.set_kpoints_mesh([1,1,1])

builder = HpBaseWorkChain.get_builder()
builder.hp.code = data.hp_code
builder.hp.hubbard_structure = data.structure
builder.hp.parameters = Dict({"INPUTHP":{"conv_thr_chi": 1e-4}})
builder.hp.qpoints = qpoints
builder.hp.parent_scf = pw_node.outputs.remote_folder

Or via the `get_builder_from_protocol`:

In [ ]:
from aiida_hubbard.workflows.hp.base import HpBaseWorkChain

builder = HpBaseWorkChain.get_builder_from_protocol(
    code=data.hp_code, # modify here if you downloaded the notebook
    protocol="fast",
    parent_scf_folder=pw_node.outputs.remote_folder,
    overrides={'hp':{'hubbard_structure':hubbard_structure}},
)

results, hp_node = run_get_node(builder)
results

:rocket: Let's inspect the results!

In [ ]:
print(HubbardUtils(results['hubbard_structure']).get_hubbard_card())

## Final considerations

We managed to compute the Hubbard parameters of LiCoO2  __fully__ ___ab initio___! :tada:
However, we had to execute quite a few steps manually, which can be tedious and error prone.
Moreover, there are the following considerations:

1. For larger and more complex structures you will need to perturb many more atoms.
   Moreover, to get converged results you will need more than one q point.
   Click [here](./2_parallel_hubbard.ipynb) to learn how to parallelize over atoms and q points.
2. To do a _full_ self-consistent calculation of these parameters, you should _relax_ your structure with the Hubbard parameters from the `hp.x` run, repeat the steps of this tutorial, relax _again_, and do this procedure over and over till convergence.
   Learn the automated way [here](./3_self_consistent.ipynb)!

:::{admonition} Learn more and in details
:class: hint

To learn the full sets of inputs, to use proficiently the `get_builder_from_protocol` and more, have a look at the following sections:
- [Specific how tos](howto/workflows/hp/base.md)
- [General information of the implemented workchain](topics/workflows/hp/base.md)
:::

:::{note}
We suggest to proceed first with the tutorial for point (1) and then the one for point (2). 
Nevertheless, tutorial (1) is not strictly necessary for (2).
:::